In [1]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.eval()



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [4]:
import json


all_spans = {}

with open("../../mfc_v4.0/spans_no_context.json", "r") as f:
    all_spans = json.load(f)

In [5]:
import torch
device = torch.device("cuda")

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback


x = []
y = []

label_map = {key:i for i,key in enumerate(all_spans.keys())}

for label in all_spans:
    for text in all_spans[label]:
        x.append(text)
        y.append(label_map[label])

print("starting tokenization")
x_tokenized = tokenizer(x,return_tensors='pt', padding=True, truncation=True, max_length=512)

print("done with tokenization")

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

print("constructing dataset")    
train_dataset = Dataset(x_tokenized, y)
print("dataset constructed")

# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model.to(device),
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

print("starting trainer")
trainer.train()
print("training complete!")

# count = 0

# n_spans = sum(len(all_spans[key]) for key in all_spans)

# batch_x = []
# batch_y = []

# model = model.to(device)

# with torch.no_grad():
#     for label in all_spans:
#         for span in all_spans[label]:
#             count += 1
#             batch_x.append(span)
#             batch_y.append(label)
#             if len(batch_x) == 60:
#                 tokenized = tokenizer(batch_x, return_tensors='pt', padding=True, max_length=512).to(device)
#                 output = model(**tokenized)
#                 x.append(output.last_hidden_state.to("cpu")[:,0,:].type(torch.float16))
#                 y.append(batch_y)
#                 del output
#                 del tokenized
#                 batch_x = []
#                 batch_y = []
#             if count % 100 == 0:
#                 print("Embedding for span %d of %d computed" % (count, n_spans))
#                 print(x[-1].shape)

starting tokenization


In [1]:
del x

NameError: name 'x' is not defined

In [10]:
torch.empty_cache()

AttributeError: module 'torch' has no attribute 'empty_cache'

In [8]:
model.zero_grad()

In [5]:
from torch import NN

class robertaClassifier(nn.Module):
    def __init__():
        pretrained = RobertaModel.from_pretrained('roberta-base')
        padder = F.pad

ImportError: cannot import name 'NN'